In [2]:
import pandas as pd
from pymongo import MongoClient
import re
from haversine import haversine

In [5]:
#!pip3 install --user haversine

In [6]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll

In [7]:
db, coll = connectCollection('companies','airports')
db, coll1 = connectCollection('companies','preferred')

In [8]:
airports = list(coll.find({"type": 'large_airport'}))

In [9]:
business = list(coll1.find({"$and":[{"money_raised":{"$ne":"$0"}},{"year":{"$gte":2009}}]}))

In [10]:
#business = list(coll1.find({"distance":{"$exists":True}}))

In [ ]:
'''
,{"money_raised":{"$regex":"k$"}}
import re
for noMatch in db.inventory.find( { "item": { "$not": re.compile("^p.*") } } ):
    print noMatch
'''

In [12]:
print(len(airports))

614


In [38]:
print(len(business))

99


In [17]:
business[0]['location']['coordinates'][0]

-73.985506

In [19]:
name=[]
for bus in business:
    name.append(bus['name'])
    

In [18]:
business[0]['location']['coordinates'][1]

40.757929

In [8]:
airports[0]

{'_id': ObjectId('5dd2e1992519448a75e6bc9a'),
 'id': 67,
 'ident': 'AYPY',
 'type': 'large_airport',
 'name': 'Port Moresby Jacksons International Airport',
 'latitude_deg': -9.443380355834961,
 'longitude_deg': 147.22000122070312,
 'elevation_ft': 146,
 'continent': 'OC',
 'iso_country': 'PG',
 'iso_region': 'PG-NCD',
 'municipality': 'Port Moresby',
 'scheduled_service': 'yes',
 'gps_code': 'AYPY',
 'iata_code': 'POM',
 'local_code': '',
 'home_link': '',
 'wikipedia_link': 'https://en.wikipedia.org/wiki/Jacksons_International_Airport',
 'keywords': '',
 'location': {'type': 'Point',
  'coordinates': [147.22000122070312, -9.443380355834961]}}

In [15]:
#long 0
airports[0]['location']['coordinates'][0]

147.22000122070312

In [16]:
#lat 1
airports[0]['location']['coordinates'][1]

-9.443380355834961

In [9]:
def getcoord(query,lat,long):
    for que in query: 
        lat.append(que['location']['coordinates'][1])
        long.append(que['location']['coordinates'][0])
    return len(lat),len(long)       

In [10]:
lat1=[]
long1=[]
getcoord(business,lat1,long1)

(99, 99)

In [11]:
bus=list(zip(lat1,long1))

In [13]:
lat2=[]
long2=[]
getcoord(airports,lat2,long2)

(614, 614)

In [14]:
air=list(zip(lat2,long2))

In [31]:
# (lat, lon)
#haversine((45.7597, 4.8422),(48.8567, 2.3508))
#243.71209416020253

In [47]:
l=[1,2,3,5]
min(l)

1

In [16]:
def harver(elem,zipi):
    comparator=[]
    for a in air:
        comparator.append(haversine(elem,a))
    return min(comparator)

In [17]:
futcol=[]
#futcol tiene que tener 99 elemento
for i in bus:
    futcol.append(harver(i,air))

In [20]:
dfair=pd.DataFrame({"name":name,"long":long1,"lat":lat1,"dis to an airport":futcol})

In [31]:
actuali=list(zip(business,futcol))

In [35]:
for i in actuali[:]:
    print(i[0])

In [36]:
for i in actuali:
    value={"$set":{'distance':i[1]}}
    coll1.update_one(i[0],value)

In [62]:
dfair

,name,long,lat,dis to an airport
0,PeekYou,-73.985506,40.757929,9.747112
1,GENWI,-111.903500,33.817100,43.734956
2,Fixya,-122.323895,37.566879,7.339122
3,Widgetbox,-122.398599,37.798853,17.835823
4,RazorGator,-118.445243,34.047312,12.149767
...,...,...,...,...
94,ticketea,-3.706176,40.445515,12.493996
95,Delphi,-83.161340,42.603532,46.262413
96,ChallengePost,-74.007170,40.740804,12.035367
97,Getyoo,4.346472,50.839970,11.848726


In [23]:
export_csv = dfair.to_csv('distoair.csv', index=None, header=True)

In [ ]:
#getAirLoc(airports[0])

In [ ]:
#for air in airports:
    #value={"$set":{'location':getAirLoc(air)}}
    #coll.update_one(air,value)   

# GEOQUERIES

In [10]:
business[0]['location']

{'type': 'Point', 'coordinates': [-73.985506, 40.757929]}

In [20]:
def geoquery(geoindex,distance,col):
    points= col.find(
        {"location": 
         {"$near": 
          {"$geometry":
           geoindex,
           "$maxDistance":distance
            }
        }
    })
    return list(points)
#que la función me devuelva la len(list(points))

In [21]:
#coll de aeropuertos
#coll1 de empresas
comparatorDosPuntoCero=[]
for bus in business:
    comparatorDosPuntoCero.append(geoquery(bus['location'],40000,coll))


In [22]:
numero=[]
for i in comparatorDosPuntoCero:
    numero.append(len(i))
print(len(numero))

99


In [17]:
#traer la columna NO SORTED
data = pd.read_csv("punctuation.csv") 
data.head()

,name,long,lat,dis to an airport,range
0,PeekYou,-73.985506,40.757929,9.747112,0-10km
1,GENWI,-111.903500,33.817100,43.734956,20-50km
2,Fixya,-122.323895,37.566879,7.339122,0-10km
3,Widgetbox,-122.398599,37.798853,17.835823,10-20km
4,RazorGator,-118.445243,34.047312,12.149767,10-20km


In [23]:
data["airports 40km"]=numero

In [24]:
data.head()

,name,long,lat,dis to an airport,range,airports 10km,airports 40km
0,PeekYou,-73.985506,40.757929,9.747112,0-10km,1,3
1,GENWI,-111.903500,33.817100,43.734956,20-50km,0,0
2,Fixya,-122.323895,37.566879,7.339122,0-10km,1,2
3,Widgetbox,-122.398599,37.798853,17.835823,10-20km,0,2
4,RazorGator,-118.445243,34.047312,12.149767,10-20km,0,1
